# TensorBoard Runs Analysis (Sequential)

This notebook loads all TensorBoard logs from `runs/`, reads all scalar tags, and plots each metric with runs stitched **sequentially** on the X axis.

In [ ]:
from pathlib import Path
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_rows', 200)

In [ ]:
RUNS_ROOT = Path('runs')
assert RUNS_ROOT.exists(), f'No runs dir found: {RUNS_ROOT.resolve()}'

event_files = sorted(RUNS_ROOT.glob('**/events.out.tfevents.*'))
run_dirs = sorted({f.parent for f in event_files}, key=lambda p: str(p))

print(f'Found {len(run_dirs)} runs')
for i, rd in enumerate(run_dirs, 1):
    print(f'{i:2d}. {rd}')

In [ ]:
def load_run_scalars(run_dir: Path):
    ea = event_accumulator.EventAccumulator(str(run_dir))
    ea.Reload()
    tags = ea.Tags().get('scalars', [])
    data = {}
    for tag in tags:
        events = ea.Scalars(tag)
        steps = np.array([e.step for e in events], dtype=np.int64)
        values = np.array([e.value for e in events], dtype=np.float64)
        data[tag] = (steps, values)
    return data

run_data = []
all_tags = set()
for rd in run_dirs:
    scalars = load_run_scalars(rd)
    run_data.append((rd, scalars))
    all_tags.update(scalars.keys())

all_tags = sorted(all_tags)
print(f'Total scalar tags: {len(all_tags)}')
print(all_tags)

In [ ]:
# Build stitched series: each run appended after previous run for each tag
stitched = {}
boundaries = {}

for tag in all_tags:
    xs = []
    ys = []
    run_marks = []
    offset = 0

    for rd, scalars in run_data:
        if tag not in scalars:
            continue
        steps, vals = scalars[tag]
        if steps.size == 0:
            continue

        # Normalize run-local steps to start from 0, then shift by global offset
        local = steps - steps.min()
        x = local + offset

        xs.append(x)
        ys.append(vals)

        run_marks.append((offset, str(rd)))
        offset = int(x.max()) + 1

    if xs:
        stitched[tag] = (np.concatenate(xs), np.concatenate(ys))
        boundaries[tag] = run_marks

print(f'Stitched tags: {len(stitched)}')

In [ ]:
rows = []
for tag in sorted(stitched.keys()):
    x, y = stitched[tag]
    rows.append({
        'tag': tag,
        'points': len(y),
        'x_min': int(x.min()) if len(x) else None,
        'x_max': int(x.max()) if len(x) else None,
        'y_min': float(np.min(y)) if len(y) else None,
        'y_max': float(np.max(y)) if len(y) else None,
        'runs_contributed': len(boundaries.get(tag, [])),
    })

summary_df = pd.DataFrame(rows).sort_values('tag').reset_index(drop=True)
summary_df

In [ ]:
# Plot all tags in a grid
tags = sorted(stitched.keys())
n = len(tags)
if n == 0:
    raise RuntimeError('No scalar tags found in runs/')

cols = 2
rows = math.ceil(n / cols)
fig, axes = plt.subplots(rows, cols, figsize=(14, 4 * rows), squeeze=False)

for i, tag in enumerate(tags):
    ax = axes[i // cols][i % cols]
    x, y = stitched[tag]
    ax.plot(x, y, linewidth=1.5)

    # Draw run boundaries
    for b, _run_name in boundaries.get(tag, [])[1:]:
        ax.axvline(b, color='gray', linestyle='--', alpha=0.25)

    ax.set_title(tag)
    ax.set_xlabel('Sequential step (runs stitched)')
    ax.set_ylabel('value')

# Hide empty subplots
for j in range(n, rows * cols):
    axes[j // cols][j % cols].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Optional: plot one specific tag
tag = 'train/sr_batch'
if tag in stitched:
    x, y = stitched[tag]
    plt.figure(figsize=(12, 4))
    plt.plot(x, y, linewidth=2.0)
    for b, _ in boundaries.get(tag, [])[1:]:
        plt.axvline(b, color='gray', linestyle='--', alpha=0.25)
    plt.title(tag + ' (runs stitched sequentially)')
    plt.xlabel('Sequential step')
    plt.ylabel('value')
    plt.tight_layout()
    plt.show()
else:
    print(f'Tag not found: {tag}')